# Test

In [1]:
import os
import sys
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO

### Instantiate ACRO

In [4]:
acro = ACRO()

DEBUG:acro:path: /Users/j4-smith/GitHub/AI-SDC/ACRO/acro/default.yaml
DEBUG:acro:config: {'output_template_file': 'ACRO output template v01b.xlsm', 'safe_SDC_set': 'Default', 'safe_tests': 'threshold nk pratio maxmin', 'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'safe_SDC_variations': 'CIS ESS', 'safe_thresholdCIS': 60, 'safe_nk_nCIS': 5, 'safe_nk_kCIS': 0.5, 'safe_testsCIS': 'threshold nk pratio', 'safe_thresholdESS': 15, 'safe_dof_thresholdESS': 10, 'safe_nk_nESS': 2, 'safe_nk_kESS': 0.9, 'safe_pratio_pESS': 0.15, 'safe_testsESS': 'nk pratio'}


### Load test data

In [5]:
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.head()

,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
0,4Children,R,1.0,2011,2880902.0,9603182.0,91404.0,310947.0,12886435.0,12127472.0,...,NaN,NaN,0.072636,0.135971,0.767809,0.094602,0.177090,0.534203,5.646843,1.0
1,4Children,R,1.0,2014,6810520.0,18768904.0,58002.0,401879.0,26039304.0,25493796.0,...,NaN,NaN,0.057641,0.089150,1.001396,0.057560,0.089026,0.646561,11.232729,1.0
2,4Children,R,1.0,2015,7199403.0,21638036.0,132191.0,512654.0,29482284.0,32290108.0,...,NaN,NaN,-0.049619,-0.079828,-0.620210,0.080004,0.128711,0.621583,7.769365,1.0
3,4Children,R,1.0,2013,5573013.0,15194731.0,228844.0,267156.0,21263744.0,20989048.0,...,NaN,NaN,0.045740,0.068251,1.008259,0.045365,0.067692,0.670166,14.772749,1.0
4,4Children,R,1.0,2010,2056816.0,7335103.0,110256.0,424628.0,9926803.0,9769816.0,...,NaN,NaN,0.057696,0.122532,0.567539,0.101660,0.215901,0.470862,4.631749,1.0


## Create a simple linear regression in three ways
 and print the results object returned and how to access the degrees of freedom
 
 I will start with inc_total vs inc_grants

### First sklearn


In [8]:
from sklearn.linear_model import LinearRegression

df.dropna(inplace=True)
y = df['inc_total'].values.reshape(-1, 1)  # values converts it into a numpy array
x = df['inc_grants'].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
linear_regressor = LinearRegression()  # create object for the class
res= linear_regressor.fit(x, y)  # perform linear regression
y_pred = linear_regressor.predict(x)  # make predictions

In [34]:
print(f'intercept {linear_regressor.intercept_}, coeffecients {linear_regressor.coef_}')

intercept [29906768.], coeffecients [[1.8979713]]


### now a linear regression with two input variables

In [36]:
x2 = df[['inc_grants','inc_activity']].values.reshape(-1, 2)  # -1 means that calculate the dimension of rows, but have 1 column
bilinear_regressor = LinearRegression()  # create object for the class
res2= bilinear_regressor.fit(x2, y)  # perform linear regression
y_pred2 = bilinear_regressor.predict(x2)  # make predictions

print(f'intercept {bilinear_regressor.intercept_}, coeffecients {bilinear_regressor.coef_}')

intercept [12081892.], coeffecients [[1.723166  1.2810115]]


### for completeness a linear regression with three input variables but no intercept

In [41]:
x3 = df[['inc_grants','inc_activity','inc_donations']].values.reshape(-1, 3)  # -1 means that calculate the dimension of rows, but have 1 column
trilinear_regressor = LinearRegression(fit_intercept=False)  # create object for the class
res3= trilinear_regressor.fit(x3, y)  # perform linear regression
y_pred3 = trilinear_regressor.predict(x3)  # make predictions


print(f'intercept {trilinear_regressor.intercept_}, coeffecients {trilinear_regressor.coef_}')

intercept 0.0, coeffecients [[1.0145854 1.0113673 1.0529848]]


### Function to calculate the degrees of freedom of a model 
(number of parameters that can vary) will depend on whether an intercept is allowed.

In [17]:
type(linear_regressor)

sklearn.linear_model._base.LinearRegression

In [29]:
import sklearn
def get_dof(model)->int:
    if isinstance( model,sklearn.linear_model._base.LinearRegression):
        dof =  model.n_features_in_ +1  if model.fit_intercept else model.n_features_in_
    return dof

In [30]:
get_dof(linear_regressor)

2

In [33]:
get_dof(bilinear_regressor)

3

In [42]:
get_dof(trilinear_regressor)

3

# TO-DO
1. Repeat this process for different ways of getting a linear regression i.e. via scipy.stats
   - in some cases we may be able to get at the dof directly if the fit process returns a results object
   - or we could consider getting at it direcrtly from the combination of the number of colu,mns in x and whether an intercept is allowed
   - either way, build up the get_dof() function
   
2. Extend to some common other forms of regression

3. use this get_dof() function within the safe framework ricahrd has created for safe_table 